# Praktikum 2, Monte Carlo
## Math modeling of the atmosphere


## Computing average autoconversion

Autoconversion is the process by which small cloud droplets grow in size and become rain drops. It can be approximated as a function of two variables, $\chi$ and $N_c$:

$$A(\chi, N_c)=H(\chi){\chi}^{\alpha}{N_c}^{\beta}$$

In [77]:
import numpy as np


def a_chi_n_c(chi, n_c, alpha, beta):
    H = lambda x: 1 if x > 0 else 0
    return H(chi) * (chi ** alpha) * (n_c ** beta)

Here $\chi$ is related to clound water mixing ratio, and $N_c$ is the number of cloud droplets. The expontent $\alpha$ is often taken to equal 2.47, and the exponent $\beta$ is often taken to equal -1.79. Therefore, if there is more cloud water or larger drops, rain forms more easily.

In [78]:
alpha = 2.47
beta = -1.79

If we integrate over this autoconversion expression over a bivariate PDF that is normal in $\chi$ and lognormal in $N_c$, we find

TODO!

1. Write a python function that computes the exact average autoconversion rate given by the formula above. As input, it should take in the means, standard deviations, and correlation of the PDF, along with the exponents $\alpha$ and $\beta$. As output, it should produce $\overline{A(\chi,N_c)}$. Use the `scipy.special` function, `pbdv`.

In [79]:
def exact_average_autoconversion_rate(my_n_c, my_chi,
                                      sigma_chi, sigma_n_c,
                                      r_chi_n_c, alpha, beta):
    import numpy as np
    from scipy.special import gamma, pbdv
    first = (1 / (np.sqrt(2 * np.pi))) * (sigma_chi ** alpha)
    second = np.exp((my_n_c * beta) + ((1 / 2) * sigma_n_c ** 2 * beta ** 2) - (
            (1 / 4) * ((my_chi / sigma_chi) + r_chi_n_c * sigma_n_c * beta) ** 2))
    third = gamma(alpha + 1)
    fourth = pbdv(-(alpha + 1), -((my_chi / sigma_chi) + r_chi_n_c * sigma_n_c * beta))[0]
    return first * second * third * fourth


2. To represent the $\chi$ distribution, create a sample from a univariate uniform distribution and transform it to a standard normal. To represent the $N_c$ distribution, create another, independent sample from a univariate standard normal. Together, these points form a sample from a bivariate,
uncorrelated standard normal distribution.

In [80]:
def draw_from_uniform_dist(n: int):
    from numpy import random
    return random.uniform(size=n)

In [81]:
def transform_to_standard_normal(points):
    from scipy.stats import norm
    return norm.ppf(points)

In [82]:
print(exact_average_autoconversion_rate(0, 0, 1, 1, 0, alpha, beta))

3.0178592303669873


In [83]:
num_points = 10


def create_chi_n_c(num_points: int):
    chi = np.array(transform_to_standard_normal(draw_from_uniform_dist(num_points)))
    n_c = np.array(transform_to_standard_normal(draw_from_uniform_dist(num_points)))
    return np.array([chi, n_c])


chi_n_c = create_chi_n_c(num_points)
print(chi_n_c)

[[ 0.29747971  1.38371926 -2.00823253  1.11673614 -0.33235746 -0.32846768
   0.94033589  0.13176406  1.1168682  -0.47566323]
 [ 0.56341103 -0.40149547  0.68780276  0.23890905 -1.8707231   1.3673392
   0.32038959 -0.83282893  1.02103946 -1.66448251]]


3. Use a Cholesky decomposition in order to transform the distribution from a 2D uncorrelated standard normal to a 2D correlated normal. Assume a correlation $r_{(\chi,N_c)n}$, a mean $N_c$ of $\mu_{N_{cn}}$ , etc.

$$
\Sigma = \begin{bmatrix}
        1 & r_{12} \\
        r_{12} & 1
        \end{bmatrix}
$$
$$
L = \begin{bmatrix}
    \sigma_x & 0 \\
    \sigma_y * r_{12} & \sigma_y\sqrt{1-{r_{12}}^2}
    \end{bmatrix}
$$

And the decomposition formular:
$$ X = L \cdot Y + \mu $$

In [84]:
def transform_to_correlated_by_cholesky(sigma_x, sigma_y, r_12, Y, my):
    import numpy as np
    L = np.array([[sigma_x, 0], [sigma_y * r_12, sigma_y * np.sqrt(1 - (r_12 ** 2))]])
    return L.dot(Y) + my

In [85]:
correlated_bivariate_dist = transform_to_correlated_by_cholesky(sigma_x=1, sigma_y=1, r_12=0.999, Y=chi_n_c, my=0)
print(correlated_bivariate_dist)

[[ 0.29747971  1.38371926 -2.00823253  1.11673614 -0.33235746 -0.32846768
   0.94033589  0.13176406  1.1168682  -0.47566323]
 [ 0.32237244  1.3643846  -1.97547252  1.12630107 -0.41566546 -0.26700523
   0.95372023  0.09439637  1.16140219 -0.54960688]]


4. In order to transform to a lognormal distribution in $N_c$, take the exponential of the $N_c$ component of the sample points.

In [86]:
def lognormal_correlated_bivariate_dist(correlated_bivariate_dist):
    return np.array(
        [correlated_bivariate_dist[0].copy(), np.exp(correlated_bivariate_dist[1])])

lognormal_correlated_bivariate_dist = lognormal_correlated_bivariate_dist(correlated_bivariate_dist)
print(lognormal_correlated_bivariate_dist)

[[ 0.29747971  1.38371926 -2.00823253  1.11673614 -0.33235746 -0.32846768
   0.94033589  0.13176406  1.1168682  -0.47566323]
 [ 1.38039879  3.91331408  0.13869576  3.08422703  0.659901    0.76566907
   2.59534702  1.09899527  3.19440931  0.57717667]]


5. Use Monte Carlo integration in order to find the integral $\overline{A(\chi,N_c)}$.

In [87]:
def A_chi_N_c_Monte_Carlo(lognormal_correlated_bivariate_dist,
                          alpha=alpha, beta=beta):
    return np.average(
        np.nan_to_num(
            [a_chi_n_c(chi=c, n_c=nc, alpha=alpha, beta=beta) for c in lognormal_correlated_bivariate_dist[0] for nc in
             lognormal_correlated_bivariate_dist[1]]))


print(A_chi_N_c_Monte_Carlo(lognormal_correlated_bivariate_dist, alpha, beta))

2.46286625537389


C:\Users\sberg\AppData\Local\Temp\ipykernel_10440\3111597490.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return H(chi) * (chi ** alpha) * (n_c ** beta)


6. Compute the Monte Carlo estimate for multiple values of N. Check whether your answer converges to the analytic answer as $\frac{1}{\sqrt{n}}$. Make sure that it converges for non-zero values of $r_{(\chi,N_c)n}$. To improve the convergence rate, choose a value of $\sigma_{N_{cn}} < 2$.

In [88]:
for n in range(10_000):
    if n % 10 == 0:

        print(f'Monte Carlo for n={n}: {A_chi_N_c_Monte_Carlo()}')

TypeError: A_chi_N_c_Monte_Carlo() missing 1 required positional argument: 'lognormal_correlated_bivariate_dist'